In [45]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

In [71]:
data = pd.read_csv('Freiwald_Tsao_faceviews_AM_data_csv/raster_data_bert_am_site013.csv')
data

label_encoder = LabelEncoder()
data = data.apply(label_encoder.fit_transform)
data

,site_info.monkey,site_info.region,labels.stimID,labels.person,labels.orientation,labels.orient_person_combo,time.1_2,time.2_3,time.3_4,time.4_5,...,time.791_792,time.792_793,time.793_794,time.794_795,time.795_796,time.796_797,time.797_798,time.798_799,time.799_800,time.800_801
0,0,0,0,0,2,50,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,2,50,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,2,50,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,1,2,61,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,1,2,61,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
679,0,0,198,23,0,16,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
680,0,0,199,24,0,17,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
681,0,0,199,24,0,17,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
682,0,0,199,24,0,17,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [73]:
data['labels.stimID'].unique()

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [75]:
data['labels.person'].unique()


array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24])

In [77]:
data['labels.orient_person_combo'].unique()

array([ 50,  61,  68,  69,  70,  71,  72,  73,  74,  51,  52,  53,  54,
        55,  56,  57,  58,  59,  60,  62,  63,  64,  65,  66,  67,  75,
        86,  93,  94,  95,  96,  97,  98,  99,  76,  77,  78,  79,  80,
        81,  82,  83,  84,  85,  87,  88,  89,  90,  91,  92, 100, 111,
       118, 119, 120, 121, 122, 123, 124, 101, 102, 103, 104, 105, 106,
       107, 108, 109, 110, 112, 113, 114, 115, 116, 117, 125, 136, 143,
       144, 145, 146, 147, 148, 149, 126, 127, 128, 129, 130, 131, 132,
       133, 134, 135, 137, 138, 139, 140, 141, 142, 150, 161, 168, 169,
       170, 171, 172, 173, 174, 151, 152, 153, 154, 155, 156, 157, 158,
       159, 160, 162, 163, 164, 165, 166, 167, 175, 186, 193, 194, 195,
       196, 197, 198, 199, 176, 177, 178, 179, 180, 181, 182, 183, 184,
       185, 187, 188, 189, 190, 191, 192,  25,  36,  43,  44,  45,  46,
        47,  48,  49,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,
        37,  38,  39,  40,  41,  42,   0,  11,  18,  19,  20,  2

In [79]:
data['labels.orientation'].unique()

array([2, 3, 4, 5, 6, 7, 1, 0])

In [81]:
y = data['labels.person']
X = data.drop(['labels.person'], axis=1)

In [83]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=11)

logreg = LogisticRegression()
logreg.fit(X_train, y_train)

Y_pred = logreg.predict(X_test)
err = 1 - accuracy_score(y_test, Y_pred)
print(f'Error rate: {err*100:.2f}%')

Error rate: 89.81%


/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
